# This colab
- Takes the Dataframe processed earlier
- embeddes 'product-description" into Chroma Vector DB
- Adds the following as metadata
- Adds the image ids s id

# Install Necessay stuff

In [ ]:
!pip install langchain_community
!pip install chromadb
!pip install bitsandbytes

# Ref: https://python.langchain.com/docs/integrations/vectorstores/chroma/
!pip install -qU langchain-huggingface

!pip install -qU "langchain-chroma>=0.1.2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings('ignore')



In [ ]:
# mount storage
from google.colab import drive
drive.mount('/content/drive')
#%cd "/gdrive/MyDrive/Interview Kickstart/MLSwitchup/Capstone/ShopTalk"
%cd /content/drive/MyDrive/ik-ml/capstone
%ls

Mounted at /content/drive
/content/drive/MyDrive/ik-ml/capstone
 11-27-brahm-ShopTalkEDA_local.ipynb   old/
 ABO_dataset/                          openai-api-key.gdoc
 chroma/                               rag-imges.ipynb
 chroma_langchain_db/                  st-embeddings-and-vector-db.ipynb
 data-old/                             st-llm.ipynb
'higgingface-access-token=.gdoc'       st-vecotr-db.ipynb
 llama32/                              st-vector-db-02.ipynb


In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from tqdm import tqdm

class VectorStore ():
  def __init__(self):
    self.preprocessed_data = None
    self.preprocessed_data_file = None
    self.content_col_name = None
    self.metadata_columns = None
    self.vector_store = None
    self.collection_name = None
    self.db_file_path = None
    self.embeddings = None
    self.retriever = None


  # Takes the preprocessed dataframe saved as pickle file
  def loadPreprocessedData(self, pre_processed_data_file, content_col_name):
    self.preprocessed_data_file = pre_processed_data_file
    self.preprocessed_data = pd.read_pickle(self.preprocessed_data_file)
    self.preprocessed_data.dropna(subset=[content_col_name], inplace=True)
    return True

  def getPreprocessedData(self):
    return self.preprocessed_data

  # if the data file exists, DB will be created from the file. if not, new DB is created
  def createVectorStore(self, collection_name, db_file_path, embeddings):
    self.collection_name = collection_name
    self.db_file_path = db_file_path
    self.embeddings = embeddings

    self.vector_store = Chroma(collection_name=self.collection_name, embedding_function=self.embeddings,
                        persist_directory=self.db_file_path,  # Where to save data locally, remove if not necessary
                    )
    return True

  def getVectorStore(self):
    return self.vector_store


  def addDataToVectorStoreIteratively(self, content_col_name, metadata_cols_list):
    self.content_col_name = content_col_name
    self.metadata_columns = metadata_cols_list

    total_count = self.preprocessed_data.shape[0]
    print('Total Records to add {}'.format(total_count))

    # convert the metadata columns
    mdf = self.preprocessed_data[self.metadata_columns]
    count = 0
    for index, row in enumerate(tqdm(self.preprocessed_data.itertuples(), total=total_count)):
      try:
        self.vector_store.add_texts(
            embedding_function=self.embeddings,
            collection_name=self.collection_name,
            ids=row.Index,
            texts = getattr(row, self.content_col_name),
            metadatas=[getattr(row, col) for col in metadata_cols_list]
        )
      except Exception as e:
        print(e)

    return True


  def getRetriever(self):
    if ( self.retriever is None):
      self.retriever = self.vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5})
    return self.retriever

  def addDataToVectorStoreBulk(self, content_col_name, metadata_cols_list):
    self.content_col_name = content_col_name
    self.metadata_columns = metadata_cols_list

    # convert the metadata columns
    mdf = self.preprocessed_data[self.metadata_columns]
    # Convert the product descriptions to strings before adding to ChromaDB
    # The tolist() method converts the Pandas Series into a list for ChromaDB
    self.vector_store.add_texts(
        embedding_function=self.embeddings,
        collection_name=self.collection_name,
        # if the df indexes are to be used as ids.if column content, then this requires change
        ids=self.preprocessed_data.index.tolist(),
        texts=self.preprocessed_data[content_col_name].astype(str).tolist(), # Convert to strings
        metadatas=mdf[metadata_cols_list].to_dict(orient="records") ,
    )
    return True



In [ ]:
# Test the class with dummay data
vs = VectorStore()
content_col_name = "product_description"
metadata_cols_list = ['brand', 'color']
ids_list = ['main_image_id']
vs.loadPreprocessedData("./ABO_dataset/abo-listings-english-tags.pkl", "product_description")
data = vs.getPreprocessedData()


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2522 entries, B08B85K5C2 to B07VF9MJ27
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   brand                2521 non-null   object
 1   bullet_point         2459 non-null   object
 2   color                1869 non-null   object
 3   fabric_type          281 non-null    object
 4   item_name            2522 non-null   object
 5   item_weight          1133 non-null   object
 6   model_name           287 non-null    object
 7   product_type         2522 non-null   object
 8   style                575 non-null    object
 9   main_image_id        2510 non-null   object
 10  other_image_id       2315 non-null   object
 11  country              2522 non-null   object
 12  marketplace          2522 non-null   object
 13  domain_name          2522 non-null   object
 14  material             1017 non-null   object
 15  item_keywords        2272 non-null   object
 

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vs.createVectorStore("st_pd", "./chroma_langchain_db", embeddings)


True

In [ ]:
vs.addDataToVectorStoreIteratively("product_description", ['brand', 'color'])

Total Records to add 2522


  0%|          | 0/2522 [00:00<?, ?it/s]

index: B08B85K5C2
metadata:  ['UMI', 'Blue']
context: {} Protect your canine companions from external elements with this comfortable dog raincoat! Designed with love for your furry friends, our fabulous dog raincoat is a protective gear that is comfortable to wear! This raincoat features a PA coating which provides durable water repellent. We all know the details really matter when it comes to a functional garment like this raincoat. There is a leash port on the back that just makes your days better! With great attention to the details, reflective tapes on the front and back are to ensure safety and visibility during walks at night. There have adhesive tapes on the neckline and underneath dogs' belly for a perfect fit. Dog lovers, what are you waiting for? Let's get one NOW!


  0%|          | 1/2522 [00:10<7:20:01, 10.47s/it]

Expected metadata to be a dict or None, got str as metadata in upsert.
count: 1
index: B005P32550
metadata:  ['Amazon Collection', 'white']
context: {} All our diamond suppliers confirm that they comply with the Kimberley Process to ensure that their diamonds are conflict free. Imported Carat weight listed is the total for all stones.


...

  0%|          | 11/2522 [00:30<54:55,  1.31s/it]  

Expected metadata to be a dict or None, got str as metadata in upsert.
count: 11
index: B07PGKCLNY
metadata:  ['365 by Whole Foods Market', nan]
context: {} ['365 일상 가치 케이지 프리 엑스트라 라지 브라운 등급 계란은 인간적으로 수확한 계란으로 다음 식사를 할 때 마음의 평화를 제공합니다. 좋아하는 디저트, 저녁 식사 조리법이나 건강한 아침 식사를 위해 이 계란을 사용하시고 이 상자에는 12개의 케이지 프리 엑스트라 라지 브라운 등급 계란이 포함되어 있다는 것을 잘 알고 있습니다. 우리의 가금류는 동물성 부산물이 없는 채식주의 식단을 공급하며 항생제 없이 자연스럽게 자랍니다.', '365 天无礼盒超大号棕色级鸡蛋是人类收获的鸡蛋，让您平静地一餐。 在您最喜爱的甜点、晚餐食中使用这些鸡蛋，或者在早餐时健康，知道这款纸箱包含 12 个无笼的超大棕色级鸡蛋。 我们的家禽为蔬菜食制食物提供食物，不含动物副产品，并且天然地营养，没有抗生素。', '365日常價值籠免費超大棕色級雞蛋是人性地收穫的雞蛋，給你安心與你的下一頓飯。 使用這些雞蛋在您最喜愛的甜點、晚餐食譜或健康早餐，並感覺很好知道這個紙箱含有12個無籠子超大的棕色級雞蛋。 我們的家禽飼養素食，不含動物副產品，天然養殖沒有抗生素。', 'Los huevos de grado marrón extra grande son huevos cosechados humanamente, lo que te da tranquilidad con tu próxima comida. Utiliza estos huevos en tus postres favoritos, recetas de cena o para un desayuno saludable y siéntete bien sabiendo que esta caja contiene 12 huevos de grado marrón extra grande sin jaulas. Nuestras aves de corral se

  0%|          | 11/2522 [00:44<2:48:26,  4.02s/it]

Expected metadata to be a dict or None, got str as metadata in upsert.
count: 12
Total count is 12


True

In [ ]:
retriever = vs.getRetriever()

In [ ]:
retriever.invoke("Give me all brands")
#retriever.invoke("Give me white color shoes list", filter={"source": "news"})

[Document(metadata={'brand': 'AmazonBasics'}, page_content='An Amazon Brand')]